## Final Project Submission

Please fill out:
* Student name: Lucas Wilkerson
* Student pace: Flex/part time
* Scheduled project review date/time: 
* Instructor name: Morgan Jones
* Blog post URL:


# Home Price Analysis 

In [ ]:
# Add Graphic 

## Project Overview 

For this project, I sought to explore how certain housing characteristics influence the price of a home in King county. Specifically, I used linear regression to show how specific housing characteristics impact the price of the home.

## Business Problem and Stakeholder

A real estate company in King County wants to increase customer acquisition and retention by providing transparent and useful information regarding the sales prices of homes. Utilizing this data, they can assist customers/clients that are both looking to buy or sell a home, understand what to budget for with a new home or what to expect to sell their current home for. For the project specifically we will be exploring which housing characteristics are the most important and have the biggest impact on sales price.

## Data Understanding 

In [2]:
# Importing standard Packages 
import pandas as pd
import numpy as np
import math 

# Importing packages for visualization 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Importing packages for statistics
import statsmodels.api as sm
from scipy import stats
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

## Data Preparation

In [3]:
#Loading dataset
kc_house_df = pd.read_csv("data/kc_house_data.csv")
kc_house_df.head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,...,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long
0,7399300360,5/24/2022,675000.0,4,1.0,1180,7140,1.0,NO,NO,...,PUBLIC,1180,0,0,40,1969,0,"2102 Southeast 21st Court, Renton, Washington ...",47.461975,-122.190520
1,8910500230,12/13/2021,920000.0,5,2.5,2770,6703,1.0,NO,NO,...,PUBLIC,1570,1570,0,240,1950,0,"11231 Greenwood Avenue North, Seattle, Washing...",47.711525,-122.355910
2,1180000275,9/29/2021,311000.0,6,2.0,2880,6156,1.0,NO,NO,...,PUBLIC,1580,1580,0,0,1956,0,"8504 South 113th Street, Seattle, Washington 9...",47.502045,-122.225200
3,1604601802,12/14/2021,775000.0,3,3.0,2160,1400,2.0,NO,NO,...,PUBLIC,1090,1070,200,270,2010,0,"4079 Letitia Avenue South, Seattle, Washington...",47.566110,-122.290200
4,8562780790,8/24/2021,592500.0,2,2.0,1120,758,2.0,NO,NO,...,PUBLIC,1120,550,550,30,2012,0,"2193 Northwest Talus Drive, Issaquah, Washingt...",47.532470,-122.071880
5,2807100156,7/20/2021,625000.0,2,1.0,1190,5688,1.0,NO,NO,...,PUBLIC,1190,0,300,0,1948,0,"1602 North 185th Street, Shoreline, Washington...",47.763470,-122.340155
6,5122400111,11/17/2021,1317227.0,3,3.0,2080,27574,1.0,NO,NO,...,PRIVATE,2080,0,0,150,1951,0,"2633 Southwest 164th Place, Burien, Washington...",47.455470,-122.367220
7,7137850210,4/28/2022,820000.0,3,2.5,2214,3506,2.0,NO,NO,...,PUBLIC,2214,0,440,206,2019,0,"24913 122nd Place Southeast, Kent, Washington ...",47.378355,-122.178625
8,2944500680,3/17/2022,780000.0,4,2.5,2340,8125,2.0,NO,NO,...,PUBLIC,2340,0,440,70,1989,0,"2721 Southwest 343rd Place, Federal Way, Washi...",47.293770,-122.369320
9,2619950340,6/21/2021,975000.0,4,2.5,2980,5859,2.0,NO,NO,...,PUBLIC,2980,0,540,170,2011,0,"27950 Northeast 147th Circle, Duvall, Washingt...",47.733170,-121.965305


In [4]:
kc_house_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30155 entries, 0 to 30154
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             30155 non-null  int64  
 1   date           30155 non-null  object 
 2   price          30155 non-null  float64
 3   bedrooms       30155 non-null  int64  
 4   bathrooms      30155 non-null  float64
 5   sqft_living    30155 non-null  int64  
 6   sqft_lot       30155 non-null  int64  
 7   floors         30155 non-null  float64
 8   waterfront     30155 non-null  object 
 9   greenbelt      30155 non-null  object 
 10  nuisance       30155 non-null  object 
 11  view           30155 non-null  object 
 12  condition      30155 non-null  object 
 13  grade          30155 non-null  object 
 14  heat_source    30123 non-null  object 
 15  sewer_system   30141 non-null  object 
 16  sqft_above     30155 non-null  int64  
 17  sqft_basement  30155 non-null  int64  
 18  sqft_g

Above I have read in the file and previewed the head of the dataset along with utilizing .info() to get a general idea of the dataset. We have a total of over 30,000 data points with very low amounts of missing values. With low amounts of missing values, I plan to drop all rows with missing values. It looks like we have 25 columns that have types that include integers, floats, and objects. Looking at the columns and data types it seems we have several numerical values and catergorical values. I may need to transform some variables but that will be determined as I explore more into the data.

In [5]:
#Dropping all rows with null values and dropping duplicate values/rows
kc_house_df.dropna(inplace= True)
kc_house_df = kc_house_df.drop_duplicates()
kc_house_df.drop(["id"], axis = 1, inplace = True)

kc_house_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30110 entries, 0 to 30154
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           30110 non-null  object 
 1   price          30110 non-null  float64
 2   bedrooms       30110 non-null  int64  
 3   bathrooms      30110 non-null  float64
 4   sqft_living    30110 non-null  int64  
 5   sqft_lot       30110 non-null  int64  
 6   floors         30110 non-null  float64
 7   waterfront     30110 non-null  object 
 8   greenbelt      30110 non-null  object 
 9   nuisance       30110 non-null  object 
 10  view           30110 non-null  object 
 11  condition      30110 non-null  object 
 12  grade          30110 non-null  object 
 13  heat_source    30110 non-null  object 
 14  sewer_system   30110 non-null  object 
 15  sqft_above     30110 non-null  int64  
 16  sqft_basement  30110 non-null  int64  
 17  sqft_garage    30110 non-null  int64  
 18  sqft_p

Duplicates and null values were dropped along with the id column as this column is not needed. Called .info() to confirm changes. We now have a total of 30,110 entries regarding home sales with 24 columns. 

In [6]:
#Inspecting value counts for all variables
for col in kc_house_df:
    print(kc_house_df[col].value_counts(), "\n")

7/1/2021      196
8/2/2021      186
6/23/2021     176
7/6/2021      176
6/16/2021     174
             ... 
12/5/2021       1
12/25/2021      1
1/16/2022       1
12/26/2021      1
6/9/2022        1
Name: date, Length: 365, dtype: int64 

650000.0     345
750000.0     306
800000.0     298
850000.0     296
600000.0     295
            ... 
903888.0       1
576200.0       1
1081500.0      1
2698000.0      1
720902.0       1
Name: price, Length: 5199, dtype: int64 

3     12745
4      9591
2      3925
5      2794
6       498
1       381
7        80
0        39
8        38
9        14
10        3
13        1
11        1
Name: bedrooms, dtype: int64 

2.5     8471
2.0     7343
1.0     4556
3.0     4116
3.5     2264
1.5     1807
4.0      644
4.5      531
5.0      145
5.5      102
6.0       45
0.0       25
6.5       25
7.5       12
7.0       12
0.5        5
9.5        2
8.0        2
8.5        1
10.0       1
10.5       1
Name: bathrooms, dtype: int64 

1250    192
1400    190
1560    183
1900 

A couple observations that were found were that there were some homes with 0 bedrooms and also 0 or 0.5 bathrooms. When looking at homes we would expect a home to have at least 1 bedroom and 1 full bathroom. To further prepare the data I will gather data from rows where homes have at least 1 bedroom and 1 full bathroom.  

In [9]:
# Dropping rows with bathrooms and bedrooms less than 1
kc_house_df = kc_house_df[kc_house_df["bedrooms"] >= 1]
kc_house_df = kc_house_df[kc_house_df["bathrooms"] >= 1]
print(kc_house_df["bedrooms"].value_counts(), kc_house_df["bathrooms"].value_counts())

print(kc_house_df.info())

3     12743
4      9588
2      3924
5      2794
6       498
1       378
7        80
8        38
9        14
10        3
13        1
11        1
Name: bedrooms, dtype: int64 2.5     8471
2.0     7342
1.0     4541
3.0     4116
3.5     2264
1.5     1806
4.0      643
4.5      531
5.0      145
5.5      102
6.0       45
6.5       25
7.0       12
7.5       12
9.5        2
8.0        2
8.5        1
10.0       1
10.5       1
Name: bathrooms, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30062 entries, 0 to 30154
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           30062 non-null  object 
 1   price          30062 non-null  float64
 2   bedrooms       30062 non-null  int64  
 3   bathrooms      30062 non-null  float64
 4   sqft_living    30062 non-null  int64  
 5   sqft_lot       30062 non-null  int64  
 6   floors         30062 non-null  float64
 7   waterfront     30062 non-null  object 
 

## Data Analysis

In [10]:
# Finding which factors correlate most with price
kc_house_df.corr()["price"]

price            1.000000
bedrooms         0.293157
bathrooms        0.483939
sqft_living      0.612011
sqft_lot         0.085546
floors           0.182296
sqft_above       0.541596
sqft_basement    0.245927
sqft_garage      0.265115
sqft_patio       0.315132
yr_built         0.096709
yr_renovated     0.085677
lat              0.063650
long            -0.022337
Name: price, dtype: float64

Looking at the correlations of the current numeric variables we see that the top 3 highest correlates are:
- Sqft_living: highest correlation at 0.61
- Sqft_above: 2nd highest correlation at 0.54
- Bathrooms (# of bathrooms): 3rd highest correlation of 0.48

Other noteable characteristics correlations include:
- Bedrooms (# of bedrooms): correlation of 0.29
- Floors (number of floors/levels): correlation of 0.18
- Yr_built: correlation of 0.096

Square footage having the highest correlation does seem appropropriate. Surprisingly number of bedrooms was not among the highest charactristics regarding correlation, however it does show a positive but weak correlation. 

In [11]:
# Creating a subset to further explore sqft_living, floors, bathrooms, bedrooms with slaes price
kc_subset = kc_house_df[['sqft_living', 'floors', 'bathrooms','bedrooms', 'price']].copy()
kc_subset

,sqft_living,floors,bathrooms,bedrooms,price
0,1180,1.0,1.0,4,675000.0
1,2770,1.0,2.5,5,920000.0
2,2880,1.0,2.0,6,311000.0
3,2160,2.0,3.0,3,775000.0
4,1120,2.0,2.0,2,592500.0
...,...,...,...,...,...
30150,1910,1.5,2.0,5,1555000.0
30151,2020,2.0,2.0,3,1313000.0
30152,1620,1.0,2.0,3,800000.0
30153,2570,2.0,2.5,3,775000.0


## Modeling 

## Regression Results 

## Conclusion/ Recommendations

## Limitations